In [ ]:
import pandas as pd

In [1]:
from WorkforceSentimentMonitoring.data import get_prepaired_data

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Dropping initial text columns...
Identifying entries in other languages...
